In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Dense, Convolution2D, MaxPooling2D, Input, BatchNormalization, GlobalAveragePooling2D
from keras.layers import ReLU
from keras import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras2onnx
from keras2onnx import convert_keras
from onnx import save_model
import numpy as np
from tensorflow.python.keras import backend as K

# 接続されているGPUを確認

In [2]:
from tensorflow.python.client import device_lib
deviceStr = str(device_lib.list_local_devices())
isGPU = False
if "GPU" in deviceStr:
    isGPU = True

print("*** isGPU == {} ***".format(isGPU))
print(deviceStr)

*** isGPU == True ***
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14438684547275352623
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12144340115158744076
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 46988144128
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7064941866605614399
physical_device_desc: "device: 0, name: Quadro RTX 8000, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13182623459941514631
physical_device_desc: "device: XLA_GPU device"
]


# データ生成

In [3]:
# データ取得
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train_shape = x_train.shape
x_test_shape = x_test.shape

# reshape([?, 28, 28] -> [?, 1, 28, 28])
x_train = x_train.reshape(x_train_shape[0], 1, x_train_shape[1], x_train_shape[2])
x_test = x_test.reshape(x_test_shape[0], 1, x_test_shape[1], x_test_shape[2])

# データの色情報を1次元から3次元に拡張([?, 1, 28, 28] -> [?, 3, 28, 28])
# グレースケールからカラーに拡張することになるが、色情報は同じ値としている
x_train = np.tile(x_train, (1,3,1,1))
x_test = np.tile(x_test, (1,3,1,1))

if isGPU:
    data_format = "channels_first"
    input_shape = (3, 28, 28)

else:
    data_format = "channels_last"
    input_shape = (28, 28, 3)
    x_train = np.transpose(x_train, [0,2,3,1])
    x_test  = np.transpose(x_test,  [0,2,3,1])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

num_classes = 10
# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# ネットワーク定義

In [4]:
def BN(x, gpu):
    if gpu:
        return BatchNormalization(axis=1)(x)
    else:
        return BatchNormalization(axis=-1)(x)   
    
input_layer = Input(shape=input_shape)
x = Convolution2D(32, kernel_size=(3, 3), padding="same", input_shape=input_shape, data_format=data_format)(input_layer)
x = BN(x, isGPU)
x = ReLU()(x)
x = Convolution2D(32, kernel_size=(3, 3), padding="same", data_format=data_format)(x)
x = BN(x, isGPU)
x = ReLU()(x)
x = MaxPooling2D(pool_size=(2, 2), padding="same", data_format=data_format)(x)

x = Convolution2D(128, kernel_size=(3, 3), padding="same", data_format=data_format)(x)
x = BN(x, isGPU)
x = ReLU()(x)
x  = Convolution2D(128, kernel_size=(3, 3), padding="same", data_format=data_format)(x)
x = BN(x, isGPU)
x = ReLU()(x)
x = MaxPooling2D(pool_size=(2, 2), data_format=data_format)(x)

x = Convolution2D(512, kernel_size=(3, 3), padding="same", data_format=data_format)(x)
x = BN(x, isGPU)
x = ReLU()(x)
x  = Convolution2D(512, kernel_size=(3, 3), padding="same", data_format=data_format)(x)
x = BN(x, isGPU)
x = ReLU()(x)
x = MaxPooling2D(pool_size=(2, 2), data_format=data_format)(x)

x = GlobalAveragePooling2D(data_format=data_format)(x)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(input_layer, output)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# 学習

In [5]:
history = model.fit(x_train, y_train,
                    batch_size=1024,
                    epochs=50,
                    verbose=1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')],
                    validation_data=(x_test, y_test), )

Epoch 1/50
59/59 [==============================] - 6s 98ms/step - loss: 0.3990 - accuracy: 0.8946 - val_loss: 4.3407 - val_accuracy: 0.1135
Epoch 2/50
59/59 [==============================] - 5s 80ms/step - loss: 0.0389 - accuracy: 0.9887 - val_loss: 4.2404 - val_accuracy: 0.1135
Epoch 3/50
59/59 [==============================] - 5s 80ms/step - loss: 0.0237 - accuracy: 0.9929 - val_loss: 3.3589 - val_accuracy: 0.2036
Epoch 4/50
59/59 [==============================] - 5s 81ms/step - loss: 0.0169 - accuracy: 0.9950 - val_loss: 2.8160 - val_accuracy: 0.3710
Epoch 5/50
59/59 [==============================] - 5s 81ms/step - loss: 0.0132 - accuracy: 0.9962 - val_loss: 1.4356 - val_accuracy: 0.5814
Epoch 6/50
59/59 [==============================] - 5s 81ms/step - loss: 0.0102 - accuracy: 0.9973 - val_loss: 0.6662 - val_accuracy: 0.7463
Epoch 7/50
59/59 [==============================] - 5s 81ms/step - loss: 0.0072 - accuracy: 0.9981 - val_loss: 0.3132 - val_accuracy: 0.8931
Epoch 8/50
59

# 用意した画像で正しく予測できるのかチェック

In [6]:
import  glob
import cv2
imageFilePaths = glob.glob("""OpenVINOApp\OpenVINOAppCpp\Data\Image\*.png""")

for imageFilePath in imageFilePaths:
    image = cv2.imread(imageFilePath)
    
    if isGPU:
        image = np.transpose(image, [2,0,1]) 
        image = image.reshape(1,3,28,28)
    else:
        image = image.reshape(1,28,28,3)
        
    image = image / 255.0
    outputs = model.predict(image)
    print("-------   {0}  -----------".format(imageFilePath))
    for i, output in enumerate(outputs[0]):
        print("[{0}] = {1:.4f}".format(i, output))

-------   OpenVINOApp\OpenVINOAppCpp\Data\Image\img_0.png  -----------
[0] = 1.0000
[1] = 0.0000
[2] = 0.0000
[3] = 0.0000
[4] = 0.0000
[5] = 0.0000
[6] = 0.0000
[7] = 0.0000
[8] = 0.0000
[9] = 0.0000
-------   OpenVINOApp\OpenVINOAppCpp\Data\Image\img_1.png  -----------
[0] = 0.0000
[1] = 0.9999
[2] = 0.0000
[3] = 0.0000
[4] = 0.0000
[5] = 0.0000
[6] = 0.0000
[7] = 0.0000
[8] = 0.0000
[9] = 0.0000
-------   OpenVINOApp\OpenVINOAppCpp\Data\Image\img_2.png  -----------
[0] = 0.0000
[1] = 0.0000
[2] = 1.0000
[3] = 0.0000
[4] = 0.0000
[5] = 0.0000
[6] = 0.0000
[7] = 0.0000
[8] = 0.0000
[9] = 0.0000
-------   OpenVINOApp\OpenVINOAppCpp\Data\Image\img_3.png  -----------
[0] = 0.0000
[1] = 0.0000
[2] = 0.0000
[3] = 1.0000
[4] = 0.0000
[5] = 0.0000
[6] = 0.0000
[7] = 0.0000
[8] = 0.0000
[9] = 0.0000
-------   OpenVINOApp\OpenVINOAppCpp\Data\Image\img_4.png  -----------
[0] = 0.0000
[1] = 0.0000
[2] = 0.0000
[3] = 0.0000
[4] = 1.0000
[5] = 0.0000
[6] = 0.0000
[7] = 0.0000
[8] = 0.0000
[9] = 0.

# 学習済みモデルの保存

In [7]:
# .h5で保存
model.save("mnist.h5", save_format="h5")

if isGPU:
    # .onnxで保存
    onnx_model = convert_keras(model, model.name, channel_first_inputs=[1,3,28,28])
    save_model(onnx_model, "mnist.onnx")
    
else:
    # .pbで保存
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
    
    #path of the directory where you want to save your model
    frozen_out_path = ''

    # Convert Keras model to ConcreteFunction
    full_model = tf.function(lambda x: model(x))
    full_model = full_model.get_concrete_function(
        tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
    # Get frozen ConcreteFunction
    frozen_func = convert_variables_to_constants_v2(full_model)
    frozen_func.graph.as_graph_def()
    layers = [op.name for op in frozen_func.graph.get_operations()]

    # Save frozen graph to disk
    tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                      logdir=frozen_out_path,
                      name="mnist.pb",
                      as_text=False)
    # Save its text representation
    tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                      logdir=frozen_out_path,
                      name="mnist.pbtxt",
                      as_text=True)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 49 -> 29
